In [2]:
import os

In [3]:
%pwd

'c:\\Users\\musammil\\Documents\\chicken-disease-project\\research'

In [4]:
os.chdir("../")

In [16]:
%pwd

'c:\\Users\\musammil\\Documents\\chicken-disease-project'

In [39]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [40]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [41]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])



    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks

        print("Preapare Callbakcs config:", config)


        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config
    

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Chicken-fecal-images")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE  
        )

        return training_config

In [42]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [43]:
class PrepareCallback:
    def __init__(self, config:PrepareCallbacksConfig):
        self.config = config



    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath = self.config.checkpoint_model_filepath,
            save_best_only = True
        )
    
    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [54]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    def train_valid_generator(self):
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.20
        )

        dataflow_kwargs = dict(
            target_size = tuple(self.config.params_image_size[:-1]),
            batch_size = self.config.params_batch_size,
            interpolation = 'bilinear'
        )

        valid_generator = tf.keras.preprocessing.image.ImageDataGenerator(**datagenerator_kwargs)

        self.valid_generator = valid_generator.flow_from_directory(
            directory=self.config.training_data,
            target_size=dataflow_kwargs['target_size'],
            batch_size=dataflow_kwargs['batch_size'],
            interpolation=dataflow_kwargs['interpolation'],
            subset="validation",
            shuffle=False
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                fill_mode='nearest',
                rescale=1./255,
                validation_split=0.20
            )
        else:
            train_datagenerator = valid_generator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            target_size=dataflow_kwargs['target_size'],
            batch_size=dataflow_kwargs['batch_size'],
            interpolation=dataflow_kwargs['interpolation'],
            subset="training",
            shuffle=True
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)


    def train(self, callback_list: list):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            callbacks=callback_list
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [55]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(
        callback_list=callback_list
    )

except Exception as e:
     raise e

Checking for tile atr: config\config.yaml
[2024-11-23 14:06:14,516: INFO: common: yaml file: config\config.yaml loaded succesfully]
Checking for tile atr: params.yaml
[2024-11-23 14:06:14,521: INFO: common: yaml file: params.yaml loaded succesfully]
[2024-11-23 14:06:14,523: INFO: common: Created directory at artifacts]
Preapare Callbakcs config: {'root_dir': 'artifacts/prepare_callbacks', 'tensorboard_root_log_dir': 'artifacts/prepare_callbacks/tensorboard_log_dir', 'checkpoint_model_filepath': 'artifacts/prepare_callbacks/checkpoint_model_dir/model.keras'}
[2024-11-23 14:06:14,526: INFO: common: Created directory at artifacts\prepare_callbacks\checkpoint_model_dir]
[2024-11-23 14:06:14,528: INFO: common: Created directory at artifacts\prepare_callbacks\tensorboard_log_dir]
[2024-11-23 14:06:14,531: INFO: common: Created directory at artifacts\training]
Found 78 images belonging to 2 classes.
Found 312 images belonging to 2 classes.


c:\Users\musammil\Documents\chicken-disease-project\newenv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.5244 - loss: 12.7533

c:\Users\musammil\Documents\chicken-disease-project\newenv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


19/19 ━━━━━━━━━━━━━━━━━━━━ 142s 7s/step - accuracy: 0.5242 - loss: 12.8247 - val_accuracy: 0.3906 - val_loss: 38.3170
Epoch 2/10
 1/19 ━━━━━━━━━━━━━━━━━━━━ 1:50 6s/step - accuracy: 0.6250 - loss: 23.9239

C:\Users\musammil\AppData\Local\Programs\Python\Python312\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


19/19 ━━━━━━━━━━━━━━━━━━━━ 12s 331ms/step - accuracy: 0.6250 - loss: 23.9239 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 3/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 136s 7s/step - accuracy: 0.6060 - loss: 8.9236 - val_accuracy: 0.8125 - val_loss: 0.8455
Epoch 4/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 11s 293ms/step - accuracy: 0.9375 - loss: 0.3343 - val_accuracy: 0.9286 - val_loss: 0.1632
Epoch 5/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 152s 8s/step - accuracy: 0.7875 - loss: 3.2806 - val_accuracy: 0.7969 - val_loss: 1.2923
Epoch 6/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 11s 297ms/step - accuracy: 0.8125 - loss: 0.6264 - val_accuracy: 0.7143 - val_loss: 1.4674
Epoch 7/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 136s 7s/step - accuracy: 0.7033 - loss: 5.0941 - val_accuracy: 0.7500 - val_loss: 1.8267
Epoch 8/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 11s 288ms/step - accuracy: 0.8125 - loss: 0.8933 - val_accuracy: 0.8571 - val_loss: 0.8728
Epoch 9/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 132s 7s/step - accuracy: 0.8718 - loss: 0.9012 - val_accuracy: 0.9219 -